In [2]:
%load_ext sql
import os
from sqlalchemy import create_engine

pgconfig = {
    'host': 'db',
    'port': os.environ['PG_PORT'],
    'database': os.environ['PG_DATABASE'],
    'user': os.environ['PG_USER'],
    'password': os.environ['PG_PASSWORD'],
}
dsl = 'postgresql://{user}:{password}@{host}:{port}/{database}'.format(**pgconfig)
conn = create_engine(dsl)

# MagicコマンドでSQLを書くための設定
%sql conn

## やりたいこと
+ フィボナッチ数を求める

In [15]:
%%sql
-- CTEの再帰
with recursive fib(a,b) as (
    select * from (values (0,1)) as foo
    union all
    select b, a+b
    from fib
    where b < 100
)

select a
from fib

*  postgresql://padawan:***@db:5432/dsdojo_db
12 rows affected.


a
0
1
1
2
3
5
8
13
21
34


## やりたいこと
+ uncountのようなことをやる(例: ('A', 3) -> 3つの('A', 1)に分解する

In [3]:
%%sql
drop table if exists Inventory;
create table Inventory (
    goods char(10) not null primary key,
    pieces integer not null check (pieces >= 0)
);

insert into Inventory
values ('CD-rom', 3), ('zz', 2);

*  postgresql://padawan:***@db:5432/dsdojo_db
Done.
Done.
2 rows affected.


[]

In [5]:
%%sql
WITH recursive cte(goods, pieces, one) AS
(
    SELECT goods, pieces, 1 FROM Inventory
    UNION ALL
    SELECT cte.goods, cte.pieces - 1, 1
    FROM cte
    WHERE cte.pieces > 1
)
select goods, one
from cte
order by goods;

*  postgresql://padawan:***@db:5432/dsdojo_db
5 rows affected.


goods,one
CD-rom,1
CD-rom,1
CD-rom,1
zz,1
zz,1
